<a href="https://colab.research.google.com/github/NosenkoArtem/Categorical-Encoding/blob/master/HW5_NosenkoArtem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.Импорты библиотек

In [85]:
!git clone https://github.com/huggingface/peft

fatal: destination path 'peft' already exists and is not an empty directory.


In [86]:
!pip install -r ./peft/examples/lora_dreambooth/requirements.txt

In [87]:
import os

import json

import pandas as pd
from pathlib import Path
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow

import wandb

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
DIR = '/content/drive/MyDrive/dino'

PATH_IMAGES = '/content/drive/MyDrive/dino/imgs'
PATH_PROMPT = '/content/drive/MyDrive/dino/train.csv'
PATH_DREAMBOOTH = '/content/drive/MyDrive/dreambooth/train_dreambooth.py'
PATH_OUTPUT = '/content/drive/MyDrive/dino/output'

WANDB_API_KEY= '/content/drive/MyDrive/ssh/wandbkey.txt'

with open(WANDB_API_KEY, 'r') as file:
    wandb_key = file.read().strip()

In [100]:
# для colab не работает :(
os.environ['PATH_DREAMBOOTH'] = PATH_DREAMBOOTH
os.environ['PATH_IMAGES'] = PATH_IMAGES
os.environ['PATH_OUTPUT'] = PATH_OUTPUT
os.environ['PATH_PROMPT'] = PATH_PROMPT
os.environ['WANDB_KEY'] = WANDB_API_KEY

In [89]:
wandb.login(key=wandb_key)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [90]:
run = wandb.init(
    # Set the project where this run will be logged
    project="stickers",
    # Track hyperparameters and run metadata
    config={},
)

### 2.Загружаем данные


In [57]:
# path_to_json = os.path.join(DIR, "clear_labels.json")


# with open(path_to_json, 'r') as f:
#     all_clear_captions = json.load(f)

In [77]:
prompts = pd.read_csv(PATH_PROMPT, index_col=['Unnamed: 0'])
prompts.head()

prompts['prompt'] = prompts['prompt'].apply(lambda x: 'Sticker with unusual dinosaurs' + x[0].lower() + x[1:])

In [103]:
prompts.loc[1, 'prompt']

'Sticker with unusual dinosaursthe image features a person wearing a black top hat and a dinosaur mask. The mask is designed to look like a Tyrannosaurus Rex, with a wide-open mouth and sharp teeth. The person is also making a gesture with their hands, which could be interpreted as a celebratory or enthusiastic expression. The background is blurred, but it appears to be a collage of various images, possibly to create a dynamic or artistic effect. The text "[RUS] !" suggests that the image is likely meant to be humorous or satirical, possibly referencing a phrase or a theme that is not immediately clear from the image alone.'

### 3.Запуск экспериментов

In [102]:
#  !accelerate launch '/content/drive/MyDrive/dreambooth/train_dreambooth.py' \
#   --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4"  \
#   --instance_data_dir='/content/drive/MyDrive/dino/imgs' \
#   --output_dir='/content/drive/MyDrive/dino/output' \
#   --instance_prompt_path='/content/drive/MyDrive/dino/train.csv' \
#   --num_validation_images=4 \
#   --validation_prompt='Sticker with unusual dinosaursthe wearing a black top hat' \
#   --seed=42 \
#   --resolution=512 \
#   --train_batch_size=4 \
#   --scale_lr \
#   --num_dataloader_workers=4 \
#   --lr_scheduler="constant" \
#   --use_lora \
#   --lora_r=16 \
#   --lora_alpha=32 \
#   --learning_rate=1e-4 \
#   --max_train_steps=800 \
#   --report_to='$wandb_key' \
#   --wandb_key=wandb_token \
#   --wandb_project_name='stickers'

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-02-16 19:16:03.140362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739733363.173930   49272 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739733363.184137   49272 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 19:16:03.221127: I tensorflow/core/platform

In [107]:
!accelerate launch '/content/drive/MyDrive/dreambooth/train_dreambooth.py' \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
  --instance_data_dir='/content/drive/MyDrive/dino/imgs' \
  --output_dir='/content/drive/MyDrive/dino/output' \
  --train_text_encoder \
  --num_dataloader_workers=1 \
  --resolution=512 \
  --train_batch_size=4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --use_lora \
  --lora_r=16 \
  --lora_alpha=32 \
  --lora_text_encoder_r=16 \
  --lora_text_encoder_alpha=17 \
  --learning_rate=5e-5 \
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=2000 \
  --report_to="wandb" \
  --wandb_key=wandb_key \
  --wandb_project_name="stickers" \
  --num_validation_images=10 \
  --validation_prompt="Sticker with unusual dinosaursthe wearing a black top hat"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-02-16 19:46:49.749593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739735209.794664   56774 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739735209.809059   56774 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 19:46:49.847809: I tensorflow/core/platform